# Data Cleaning

Import some basic packages

In [1]:
import pandas as pd
import numpy as np

Now let's load up the second coffee data

In [2]:
coffee_df = pd.read_csv('../data/coffee.csv', index_col=[0])

Creating a copy of the original

In [3]:
coffee = coffee_df.copy().reset_index()

Take a glimpse of the data

In [4]:
display(coffee[:3])
print('The shape of coffee is ', coffee.shape)

,all_text,name,rating,roaster,slug,region_africa_arabia,region_caribbean,region_central_america,region_hawaii,region_asia_pacific,...,aroma,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3,desc_4
0,\r\n\r\n\r\n\r\n \r\n93\r\nFlight Coffee Co.\r...,Ethiopia Deri Kochoha,93,Flight Coffee Co.,/review/ethiopia-deri-kochoha-2,1,0,0,0,0,...,9.0,8.0,9.0,9.0,8.0,NaN,"Bright, crisp, sweetly tart. Citrus medley, ca...",From the Deri Kochoha mill in the Hagere Marya...,A poised and melodic wet-processed Ethiopia co...,NaN
1,\r\n\r\n\r\n\r\n\r\n91\r\nDoi Chaang Coffee\r\...,Espresso,91,Doi Chaang Coffee,/review/espresso-14,0,0,0,0,1,...,8.0,NaN,8.0,8.0,8.0,9.0,"Evaluated as espresso. Deeply rich, sweetly ro...",Doi Chaang is a single-estate coffee produced ...,"A rich, resonant espresso from Thailand, espec...",NaN
2,\r\n\r\n\r\n\r\n \r\n95\r\nTemple Coffee and T...,Kenya Ruthaka Peaberry,95,Temple Coffee and Tea,/review/kenya-ruthaka-peaberry,1,0,0,0,0,...,9.0,8.0,9.0,10.0,8.0,NaN,"Deeply sweet, richly savory. Dark chocolate, p...",Despite challenges ranging from contested gove...,"A high-toned, nuanced Kenya cup, classic in it...",NaN


The shape of coffee is  (5124, 34)


Now we remove the columns that we are not interested in.

In [5]:
coffee.columns

Index(['all_text', 'name', 'rating', 'roaster', 'slug', 'region_africa_arabia',
       'region_caribbean', 'region_central_america', 'region_hawaii',
       'region_asia_pacific', 'region_south_america', 'type_espresso',
       'type_organic', 'type_fair_trade', 'type_decaffeinated',
       'type_pod_capsule', 'type_blend', 'type_estate', 'location', 'origin',
       'roast', 'est_price', 'review_date', 'agtron', 'aroma', 'acid', 'body',
       'flavor', 'aftertaste', 'with_milk', 'desc_1', 'desc_2', 'desc_3',
       'desc_4'],
      dtype='object')

In [6]:
Unwanted = ['all_text', 'name', 'roaster', 'slug', 'origin', 'est_price', 'location',
            'agtron', 'with_milk', 'desc_1', 'desc_2', 'desc_3', 'desc_4']

In [7]:
for categ in Unwanted:
    coffee.drop(categ, inplace = True, axis = 1)
print('The shape of coffee is ', coffee.shape)

The shape of coffee is  (5124, 21)


Determine the percentage of the missing values within each feature. 

In [8]:
cols = coffee.columns
coffee[cols].isna().sum().values/coffee[cols].shape[0]*100

df_null = pd.DataFrame({
    'null%' : coffee[cols].isna().sum().values/coffee[cols].shape[0]*100},
    index = cols)

df_null

,null%
rating,0.000000
region_africa_arabia,0.000000
region_caribbean,0.000000
region_central_america,0.000000
region_hawaii,0.000000
region_asia_pacific,0.000000
region_south_america,0.000000
type_espresso,0.000000
type_organic,0.000000
type_fair_trade,0.000000


We decide to drop `acid` and `aftertaste` since their Null rate is pretty high.

In [9]:
coffee.drop(['acid', 'aftertaste'], inplace = True, axis = 1)

Drop the rows where `roast` or `aroma` or `body` or `flavor` contain Null values.

In [10]:
coffee = coffee[coffee['roast'].notna() & coffee['aroma'].notna() & coffee['body'].notna() & coffee['flavor'].notna()]
print('The shape of coffee is ', coffee.shape)

The shape of coffee is  (4680, 19)


Separate `review_date` column into two different columns `month` and `year` and drop the original `review_date` column.

In [11]:
coffee = coffee.assign(
    month = lambda d: d.review_date.str.split(" ").str[0],
    year  = lambda d: d.review_date.str.split(" ").str[1]
)
coffee.drop(['review_date'], inplace = True, axis = 1)

Adjust the type of each feature in the data and convert one-hot encoding back to a categorical column (region).

In [12]:
coffee['region'] = (coffee.iloc[:, 1:6] == 1).idxmax(1)
to_convert = ['type_espresso', 'type_organic', 'type_fair_trade', 'type_blend',
              'type_decaffeinated', 'type_pod_capsule', 'type_estate']
coffee[to_convert] = coffee[to_convert].astype('category')
coffee['rating'] = pd.to_numeric(coffee['rating'], errors='coerce')
coffee['year'] = pd.to_numeric(coffee['year'], errors='coerce')

In [13]:
coffee.dtypes

rating                     float64
region_africa_arabia         int64
region_caribbean             int64
region_central_america       int64
region_hawaii                int64
region_asia_pacific          int64
region_south_america         int64
type_espresso             category
type_organic              category
type_fair_trade           category
type_decaffeinated        category
type_pod_capsule          category
type_blend                category
type_estate               category
roast                       object
aroma                      float64
body                       float64
flavor                     float64
month                       object
year                         int64
region                      object
dtype: object

In [14]:
coffee.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4680 entries, 0 to 5123
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   rating                  4677 non-null   float64 
 1   region_africa_arabia    4680 non-null   int64   
 2   region_caribbean        4680 non-null   int64   
 3   region_central_america  4680 non-null   int64   
 4   region_hawaii           4680 non-null   int64   
 5   region_asia_pacific     4680 non-null   int64   
 6   region_south_america    4680 non-null   int64   
 7   type_espresso           4680 non-null   category
 8   type_organic            4680 non-null   category
 9   type_fair_trade         4680 non-null   category
 10  type_decaffeinated      4680 non-null   category
 11  type_pod_capsule        4680 non-null   category
 12  type_blend              4680 non-null   category
 13  type_estate             4680 non-null   category
 14  roast                   

In [15]:
#Removes rows with null values in Rating
coffee.dropna(subset=['rating'], inplace=True)

Finally, we save the file.

In [16]:
coffee.to_csv('../data/coffee_after_cleaning.csv', index = False)

In [17]:
coffee

,rating,region_africa_arabia,region_caribbean,region_central_america,region_hawaii,region_asia_pacific,region_south_america,type_espresso,type_organic,type_fair_trade,...,type_pod_capsule,type_blend,type_estate,roast,aroma,body,flavor,month,year,region
0,93.0,1,0,0,0,0,0,0,0,0,...,0,0,0,Medium-Light,9.0,9.0,9.0,January,2019,region_africa_arabia
1,91.0,0,0,0,0,1,0,1,0,0,...,0,0,1,Medium,8.0,8.0,8.0,January,2019,region_asia_pacific
2,95.0,1,0,0,0,0,0,0,0,0,...,0,0,0,Medium,9.0,9.0,10.0,January,2019,region_africa_arabia
3,93.0,1,0,0,0,0,0,0,0,0,...,0,0,0,Medium-Light,9.0,9.0,9.0,January,2019,region_africa_arabia
4,93.0,0,0,0,0,0,0,1,0,0,...,0,0,0,Medium-Light,9.0,8.0,9.0,January,2019,region_africa_arabia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5119,83.0,0,0,0,0,0,0,0,0,0,...,0,0,0,Medium-Light,8.0,7.0,7.0,February,1997,region_africa_arabia
5120,81.0,0,0,0,0,0,0,0,0,0,...,0,0,0,Dark,7.0,6.0,8.0,February,1997,region_africa_arabia
5121,75.0,0,0,0,0,0,0,0,0,0,...,0,0,0,Medium,6.0,6.0,5.0,February,1997,region_africa_arabia
5122,74.0,0,0,0,0,0,0,0,0,0,...,0,0,0,Light,7.0,6.0,6.0,February,1997,region_africa_arabia


Now we make another file that trims more of the columns out.

In [18]:
coffee.columns

Index(['rating', 'region_africa_arabia', 'region_caribbean',
       'region_central_america', 'region_hawaii', 'region_asia_pacific',
       'region_south_america', 'type_espresso', 'type_organic',
       'type_fair_trade', 'type_decaffeinated', 'type_pod_capsule',
       'type_blend', 'type_estate', 'roast', 'aroma', 'body', 'flavor',
       'month', 'year', 'region'],
      dtype='object')

In [19]:
Unwanted_part_2 = ['aroma', 'body', 'flavor', 'month', 'year']

In [20]:
for categ in Unwanted_part_2:
    coffee.drop(categ, inplace = True, axis = 1)

In [21]:
coffee.head()

,rating,region_africa_arabia,region_caribbean,region_central_america,region_hawaii,region_asia_pacific,region_south_america,type_espresso,type_organic,type_fair_trade,type_decaffeinated,type_pod_capsule,type_blend,type_estate,roast,region
0,93.0,1,0,0,0,0,0,0,0,0,0,0,0,0,Medium-Light,region_africa_arabia
1,91.0,0,0,0,0,1,0,1,0,0,0,0,0,1,Medium,region_asia_pacific
2,95.0,1,0,0,0,0,0,0,0,0,0,0,0,0,Medium,region_africa_arabia
3,93.0,1,0,0,0,0,0,0,0,0,0,0,0,0,Medium-Light,region_africa_arabia
4,93.0,0,0,0,0,0,0,1,0,0,0,0,0,0,Medium-Light,region_africa_arabia


Now we one hot encode the roast category.

In [22]:
roasts = pd.get_dummies(coffee['roast'])

In [23]:
coffee = coffee.join(roasts)

In [24]:
coffee.head()

,rating,region_africa_arabia,region_caribbean,region_central_america,region_hawaii,region_asia_pacific,region_south_america,type_espresso,type_organic,type_fair_trade,...,type_blend,type_estate,roast,region,Dark,Light,Medium,Medium-Dark,Medium-Light,Very Dark
0,93.0,1,0,0,0,0,0,0,0,0,...,0,0,Medium-Light,region_africa_arabia,0,0,0,0,1,0
1,91.0,0,0,0,0,1,0,1,0,0,...,0,1,Medium,region_asia_pacific,0,0,1,0,0,0
2,95.0,1,0,0,0,0,0,0,0,0,...,0,0,Medium,region_africa_arabia,0,0,1,0,0,0
3,93.0,1,0,0,0,0,0,0,0,0,...,0,0,Medium-Light,region_africa_arabia,0,0,0,0,1,0
4,93.0,0,0,0,0,0,0,1,0,0,...,0,0,Medium-Light,region_africa_arabia,0,0,0,0,1,0


In [25]:
coffee.to_csv('../data/one_hot_coffee.csv', index = False)

We can also try changing the roast category to become a scale value. We will try the following.

In [26]:
#1 = Light
#2 = Medium-Light
#3 = Medium
#4 = Medium-Dark
#5 = Dark
#6 = Very Dark

coffee['roast_scale'] = coffee['roast'].replace(['Light','Medium-Light','Medium','Medium-Dark','Dark','Very Dark'],
                                          [1,2,3,4,5,6]).copy()

In [27]:
coffee.head()

,rating,region_africa_arabia,region_caribbean,region_central_america,region_hawaii,region_asia_pacific,region_south_america,type_espresso,type_organic,type_fair_trade,...,type_estate,roast,region,Dark,Light,Medium,Medium-Dark,Medium-Light,Very Dark,roast_scale
0,93.0,1,0,0,0,0,0,0,0,0,...,0,Medium-Light,region_africa_arabia,0,0,0,0,1,0,2
1,91.0,0,0,0,0,1,0,1,0,0,...,1,Medium,region_asia_pacific,0,0,1,0,0,0,3
2,95.0,1,0,0,0,0,0,0,0,0,...,0,Medium,region_africa_arabia,0,0,1,0,0,0,3
3,93.0,1,0,0,0,0,0,0,0,0,...,0,Medium-Light,region_africa_arabia,0,0,0,0,1,0,2
4,93.0,0,0,0,0,0,0,1,0,0,...,0,Medium-Light,region_africa_arabia,0,0,0,0,1,0,2


In [28]:
coffee.to_csv('../data/scaled_roast_coffee.csv', index = False)